In [63]:
import pandas as pd
import json
import gzip
import os
import requests
import pickle
import time
from tqdm import tqdm
from functools import lru_cache
from collections import Counter
from itertools import chain
pd.set_option("display.max_columns", 30)

import sys, os
sys.path.insert(0, "/home/gstupp/projects/WikidataIntegrator")
sys.path.insert(0, "/home/gstupp/projects/wikidata-biothings/scheduled_bots")
from wikidataintegrator import wdi_helpers
from scheduled_bots.drugs.chemlib import create_item
from scheduled_bots.drugs.chemspider import ChemSpiderMolecule
from scheduled_bots.drugs.unii import UNIIMolecule

from wikidataintegrator import wdi_core, wdi_login

In [64]:
try:
    from scheduled_bots.local import WDUSER, WDPASS
except ImportError:
    if "WDUSER" in os.environ and "WDPASS" in os.environ:
        WDUSER = os.environ['WDUSER']
        WDPASS = os.environ['WDPASS']
    else:
        raise ValueError("WDUSER and WDPASS must be specified in local.py or as environment variables")

login = wdi_login.WDLogin(user=WDUSER, pwd=WDPASS)

Successfully logged in as ProteinBoxBot


In [73]:
df = pd.read_csv("openfda_single.csv", index_col=0)
df.unii = df.unii.map(eval)
df.product_ndc = df.product_ndc.map(eval)
df.head()

,application_number,brand_name,generic_name,is_original_packager,manufacturer_name,nui,original_packager_product_ndc,package_ndc,pharm_class_cs,pharm_class_epc,pharm_class_moa,pharm_class_pe,product_ndc,product_type,route,rxcui,rxcui_brandname,rxcui_ingredient,spl_id,spl_set_id,substance_name,unii,upc
3318,ANDA077844,ABACAVIR,ABACAVIR SULFATE,[],AMERICAN HEALTH PACKAGING,[],['65862-073'],"['68084-021-21', '68084-021-11']",[],[],[],[],[68084-021],HUMAN PRESCRIPTION DRUG,['ORAL'],['242679'],221017,190521,5CDBCF4A-4B9B-5549-E053-2991AA0A6550,01E46F58-8BDA-4FF3-AB21-57D5B540D440,['ABACAVIR SULFATE'],[J220T4J9Q2],[]
5846,NDA021652,ABACAVIR AND LAMIVUDINE,ABACAVIR SULFATE AND LAMIVUDINE,[True],PRASCO LABORATORIES,"['N0000175459', 'N0000175656', 'N0000009947', ...",[],['66993-482-30'],['Nucleoside Analog [Chemical/Ingredient]'],['Hepatitis B Virus Nucleoside Analog Reverse ...,['Nucleoside Reverse Transcriptase Inhibitors ...,[],[66993-482],HUMAN PRESCRIPTION DRUG,['ORAL'],['602393'],497184,614534,704ACC32-038D-4B75-907D-96E4F5839EA4,133490FC-26EB-4C92-A21D-52BE4C226B74,"['LAMIVUDINE', 'ABACAVIR SULFATE']","[J220T4J9Q2, 2T8Q726O95]",[]
6415,ANDA202912,"ABACAVIR, LAMIVUDINE AND ZIDOVUDINE","ABACAVIR , LAMIVUDINE AND ZIDOVUDINE",[True],"LUPIN PHARMACEUTICALS, INC.","['N0000175459', 'N0000175656', 'N0000009947', ...",[],"['68180-286-07', '68180-286-01', '68180-286-02']",['Nucleoside Analog [Chemical/Ingredient]'],['Hepatitis B Virus Nucleoside Analog Reverse ...,['Nucleoside Reverse Transcriptase Inhibitors ...,[],[68180-286],HUMAN PRESCRIPTION DRUG,['ORAL'],['307650'],284904,284620,E1E30726-5522-4214-B103-A016A08FDFCB,F5F7C0D9-A247-4308-8269-32ACCC490EA6,"['LAMIVUDINE', 'ZIDOVUDINE', 'ABACAVIR SULFATE']","[J220T4J9Q2, 2T8Q726O95, 4B9XT59T7S]",[]
104,NDA021436,ABILIFY,ARIPIPRAZOLE,[],TYA PHARMACEUTICALS,['N0000175430'],['59148-009'],['64725-0009-1'],[],['Atypical Antipsychotic [EPC]'],[],[],[64725-0009],HUMAN PRESCRIPTION DRUG,['ORAL'],"['352308', '349490']",352393,89013,1C1E7873-4FBF-4033-94D7-9FD8416B828E,D0640208-44E9-4052-A56F-9A4CD9A5AAAB,['ARIPIPRAZOLE'],[82VFR53I78],[]
3062,NDA022510,ABSTRAL,FENTANYL CITRATE,[True],"GALENA BIOPHARMA, INC.",[],[],"['57881-334-04', '57881-333-32', '57881-332-32...",[],[],[],[],"[57881-338, 57881-331, 57881-333, 57881-332, 5...",HUMAN PRESCRIPTION DRUG,['SUBLINGUAL'],"['1053652', '1053651', '1053658', '1053657', '...",1053648,4337,621FD88B-B921-47A2-94BD-A778D6005353,F969E2BC-6297-4E29-89D3-A3685A2C7C6B,['FENTANYL CITRATE'],[MUN5LYG46H],[]


In [155]:
def get_label_from_rxcui(rxcui):
    url = "https://rxnav.nlm.nih.gov/REST/rxcui/{}/properties.json".format(rxcui)
    d = requests.get(url).json()
    if d:
        return d['properties']['name']
# rxcui_label = dict()
for rxcui in tqdm(set(df.rxcui_brandname)):
    if rxcui not in rxcui_label:
        rxcui_label[rxcui] = get_label_from_rxcui(rxcui)

100%|██████████| 1513/1513 [03:40<00:00,  6.87it/s] 


In [156]:
# get the rxnorm label for the brand_name, becuase often, the openfda version sucks
# https://rxnav.nlm.nih.gov/REST/rxcui/497184/properties.json
df['brand_name_rxnorm'] = df.rxcui_brandname.map(rxcui_label.get)

In [157]:
rxnorm_qid = wdi_helpers.id_mapper("P3345", return_as_set=True)
rxnorm_qid = {k:list(v)[0] for k,v in rxnorm_qid.items() if len(v)==1}

In [158]:
df['rxcui_brandname_qid'] = df.rxcui_brandname.map(rxnorm_qid.get)
len(df['rxcui_brandname_qid'])

2042

In [159]:
len(df['rxcui_brandname_qid'].dropna())

285

In [166]:
def search_wikidata(s):
    # search for a string
    # https://www.wikidata.org/w/api.php?action=wbsearchentities&search=ABSTRAL&language=en&type=item&format=json
    params = {'action': 'wbsearchentities',
              'language': 'en',
              'search': s,
              'type': "item",
              'format': 'json'}
    r = requests.get("https://www.wikidata.org/w/api.php", params=params)
    r.raise_for_status()
    d = r.json()
    dataPage = d['search']
    for item in dataPage:
        item['id'] = "wd:" + item['id']
        del item['repository']
        del item['concepturi']
    return dataPage

def get_sitelink(qid):
    # get sitelink
    # https://www.wikidata.org/w/api.php?action=wbgetentities&ids=Q42&props=sitelinks&format=json&sitefilter=enwiki
    params = {'action': 'wbgetentities',
              'ids': qid,
              'props': 'sitelinks',
              'sitefilter': "enwiki",
              'format': 'json'}
    r = requests.get("https://www.wikidata.org/w/api.php", params=params)
    r.raise_for_status()
    d = r.json()['entities'][qid]['sitelinks']
    if 'enwiki' in d:
        return d['enwiki']['title']

In [181]:
row = df.iloc[11]
row

application_number                                                       NDA020125
brand_name                                                               ACCURETIC
generic_name                       QUINAPRIL HYDROCHLORIDE AND HYDROCHLOROTHIAZIDE
is_original_packager                                                        [True]
manufacturer_name                                    PARKE-DAVIS DIV OF PFIZER INC
nui                                  ['N0000166469', 'N0000175419', 'N0000175359']
original_packager_product_ndc                                                   []
package_ndc                       ['0071-0222-23', '0071-0223-23', '0071-0220-23']
pharm_class_cs                                 ['Thiazides [Chemical/Ingredient]']
pharm_class_epc                                        ['Thiazide Diuretic [EPC]']
pharm_class_moa                                                                 []
pharm_class_pe                                         ['Increased Diuresis [PE]']
prod

In [182]:
brand_name = row.brand_name_rxnorm
rxcui = row.rxcui_brandname
product_ndc = row.product_ndc[0]
qid = row.rxcui_brandname_qid # qid from rxcui_brandname

if not qid:
    # maybe there is an existing item. search by brand name label
    r = search_wikidata(brand_name)
    if r:
        r = r[0]
        descr = r.get("description")
        print(brand_name)
        print(r.get("label"))
        if brand_name.lower() == r.get("label").lower():
            if descr and descr not in {"pharmaceutical product"}:
                print(descr)
                cont=input()
                if cont == "y":
                    qid = r['title']
                    row.rxcui_brandname_qid = qid
            else:
                qid = r['title']
                row.rxcui_brandname_qid = qid

Accuretic
Accuretic


In [183]:
qid

'Q4672916'

In [184]:
refs = [[
    wdi_core.WDItemID(value='Q22907487', prop_nr='P248', is_reference=True),  # stated in OpenFDA
    wdi_core.WDExternalID(value=product_ndc, prop_nr='P3640', is_reference=True),  # product_ndc
    wdi_core.WDTime(time=time.strftime('+%Y-%m-%dT00:00:00Z'), prop_nr='P813', is_reference=True)  # retrieved
]]
# rxnorm
data = [wdi_core.WDString(prop_nr="P3345", value=rxcui, references=refs)]

In [185]:
if qid:
    item = wdi_core.WDItemEngine(wd_item_id=qid, domain='drugs', data=data, append_value=['P31'])
    if brand_name.lower() != item.get_label().lower() and brand_name.lower() not in map(str.lower, item.get_aliases()):
        item.set_aliases([brand_name], append=True)
else:
    data.append(wdi_core.WDItemID(value='Q28885102', prop_nr='P31'))  # pharmaceutical product
    item = wdi_core.WDItemEngine(item_name='drug', domain='drugs', data=data, append_value=['P31'])
    item.set_label(brand_name)
if not item.get_description():
    item.set_description("pharmaceutical product")

In [186]:
item.write(login)

'Q4672916'